In [129]:
from dateutil.relativedelta import relativedelta
import pandas as pd
import datetime
from pandas.tseries.offsets import BMonthEnd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import yfinance as yf



In [130]:
def back_test_buy_and_hold(df,singal_col,ret_col1,holding=None):
    position = []
    if holding == None:
        for i in range(len(df)):
            factor_value = df[singal_col][i]
            if factor_value:
                position.append(1)
            else:
                position.append(0)
    else:
        count = holding
        for i in range(len(df)):
            factor_value = df[singal_col][i]
            if factor_value:
                position.append(1)
                count = 1
            else:
                if count < holding:
                    count += 1
                    position.append(1)
                else:
                    position.append(0)
    
    df['position'] = position
    df['strat_ret']=df['position']*df[ret_col1]# t-1 close entry
    df['cum_ret_'+"buy&hold"]=np.exp(df[ret_col1].cumsum()) - 1
    df['cum_ret_strat']=np.exp(df['strat_ret'].cumsum()) - 1
    df['peak_'+"buy_hold"] = df['cum_ret_'+"buy&hold"].cummax()
    df['strat_peak'] = df['cum_ret_strat'].cummax()
    
    return df

def getStratStats_single(log_returns: pd.Series, 
    risk_free_rate: float = 0.02):
    stats = {}  
   
    stats['tot_ret'] = np.exp(log_returns.sum()) - 1
 
    stats['annual_ret'] = np.exp(log_returns.mean() * 240) - 1
   
    stats['annual_vol'] = log_returns.std() * np.sqrt(240)
   
    annualized_downside = log_returns.loc[log_returns<0].std() *  np.sqrt(240)
    stats['sortino_ratio'] = (stats['annual_ret'] -  risk_free_rate) / annualized_downside  
   
    stats['sharpe_ratio'] = (stats['annual_ret'] - risk_free_rate) / stats['annual_vol']  
   
    cum_returns = np.exp(log_returns.cumsum()) - 1
    peak = cum_returns.cummax()
    drawdown = peak - cum_returns
    stats['max_drawdown'] = drawdown.max()  
   
    strat_dd = drawdown[drawdown==0]
    strat_dd_diff = strat_dd.index[1:] - strat_dd.index[:-1]
    strat_dd_days = strat_dd_diff.map(lambda x: x.days).values
    strat_dd_days = np.hstack([strat_dd_days,
    (drawdown.index[-1] - strat_dd.index[-1]).days])
    stats['max_drawdown_duration'] = strat_dd_days.max()
    return stats

In [173]:
stock=yf.Ticker('^TWII').history(start="1990-01-01",  end="2021-12-30")
stock['ret']=np.log(stock.Close/stock.Close.shift(1))
stock=stock[['ret']]
trade_date=[i for i in set(stock.index)]
trade_date.sort()
t_trade_date=pd.DataFrame({'date':list(filter(lambda x: (x.day<=15), trade_date))})
t_trade_date=pd.DataFrame({'date':list(test.groupby([test.date.dt.year, test.date.dt.month]).last()['date']),'mark':True})

cli=pd.read_csv('https://stats.oecd.org/sdmx-json/data/DP_LIVE/.CLI.../OECD?contentType=csv&detail=code&separator=comma&csv-lang=en')[['LOCATION','TIME','Value']]
cli['TIME'] =  pd.to_datetime(cli['TIME'], format='%Y/%m')
cli['TIME']=[i + relativedelta(months=1)-relativedelta(days=1) for i in cli['TIME'] ]#set month end date
cli=cli.sort_values(by=['LOCATION','TIME'])

countries=list(set(cli['LOCATION']))
total=cli.groupby("TIME").count()['Value']
cli=cli.pivot(index="TIME", columns='LOCATION', values='Value') 
cli['total']=total
cli['diffusion_index'] = 0
for c in countries:
    cli.loc[cli[c].diff()>0., 'diffusion_index'] += 1
cli['diffusion_index'] = 1.*cli['diffusion_index']/cli['total']

# count of month(diff>0.5)
cli['CC50'] = 0
for i in range(0,18):
    cli.loc[(cli['diffusion_index']>=0.5) & (cli['diffusion_index'].shift(i)>=0.5), 'CC50'] += 1 ##larger=more time >0.5


SIGNALS = cli.loc[(cli['CC50']==0) & (cli['CC50'].shift(1)>=5)] #exp end(temp)

SIGNALS['to_keep']=False
SIGNALS.iloc[0,46]=True#beg
last_kept_date = SIGNALS.index[0]

for i in SIGNALS.index:
    if (i - last_kept_date).days > 365*15/12.:# total days(15 month)
        last_kept_date = i#new end
        SIGNALS.loc[i, 'to_keep'] = True #real exp end
        continue

SIGNALS = SIGNALS.loc[SIGNALS['to_keep']==True]#turning point
SIGNALS = SIGNALS['1959':] 
SIGNALS=SIGNALS.reset_index()
SIGNALS=SIGNALS[['TIME','to_keep']]

df=pd.merge(t_trade_date,SIGNALS, left_on=[t_trade_date['date'].apply(lambda x: (x.month)),t_trade_date['date'].apply(lambda x: (x.year))],right_on=[SIGNALS['TIME'].apply(lambda y: (y.month+1)),SIGNALS['TIME'].apply(lambda y: (y.year))],how='left' ).drop(['key_1','key_0','mark','TIME'], 1)
df=df[df['to_keep']==True]
df=df.set_index('date')
stock=stock.dropna()
stock

C:\Users\antho\AppData\Local\Temp/ipykernel_26768/3082753612.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SIGNALS['to_keep']=False
C:\Users\antho\anaconda3\envs\py38_zip\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
C:\Users\antho\anaconda3\envs\py38_zip\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[ro

,ret
Date,
1997-07-03,0.003430
1997-07-04,0.018106
1997-07-07,0.014072
1997-07-08,-0.001886
1997-07-09,0.006151
...,...
2021-12-23,0.006699
2021-12-24,0.000834
2021-12-27,0.004849


In [183]:
df.index

DatetimeIndex(['1997-11-14', '2000-06-15', '2002-06-14', '2004-11-15',
               '2007-08-15', '2011-03-15', '2014-05-15', '2018-03-15',
               '2021-11-15'],
              dtype='datetime64[ns]', name='date', freq=None)

In [178]:
stock_sub=stock.loc[df.index[0]:str(df.index[0].year+5)+"-"+"12-31"]
stock_sub=stock_sub.join(df)
stock_sub['to_keep']=stock_sub['to_keep'].fillna(False)#true=signal day

day=[10,20,30,40,50,60,70,80,90,100]
result=pd.DataFrame()
for i in day:
    result=result.append( pd.DataFrame([getStratStats_single(back_test_buy_and_hold(stock_sub,'to_keep','ret',holding=i)['strat_ret'])],index=['Hold_'+str(i)]))
result=result.append( pd.DataFrame([getStratStats_single(back_test_buy_and_hold(stock_sub,'to_keep','ret',holding=i)['ret'])],index=['Buy&Hold']))
result

,tot_ret,annual_ret,annual_vol,sortino_ratio,sharpe_ratio,max_drawdown,max_drawdown_duration
Hold_10,-0.122891,-0.024725,0.030499,-0.338000,-1.466428,0.151801,1866
Hold_20,-0.076384,-0.015057,0.063233,-0.185997,-0.554407,0.243358,1847
Hold_30,-0.130033,-0.026246,0.075448,-0.277304,-0.612951,0.268657,1847
Hold_40,-0.243069,-0.051782,0.091091,-0.374404,-0.788024,0.389225,1847
Hold_50,-0.094422,-0.018759,0.098061,-0.209249,-0.395248,0.291575,1792
Hold_60,-0.220075,-0.046349,0.106671,-0.374237,-0.621993,0.389819,930
Hold_70,-0.299770,-0.065775,0.117210,-0.467636,-0.731800,0.527720,1765
Hold_80,-0.383761,-0.088290,0.126636,-0.565491,-0.855129,0.611711,1765
Hold_90,-0.397144,-0.092104,0.144824,-0.535765,-0.774072,0.707952,1765
Hold_100,-0.393604,-0.091089,0.153345,-0.549929,-0.724437,0.718611,1765


In [179]:
stock_sub=stock.loc[df.index[1]:str(df.index[1].year+5)+"-"+"12-31"]
stock_sub=stock_sub.join(df)
stock_sub['to_keep']=stock_sub['to_keep'].fillna(False)#true=signal day

day=[10,20,30,40,50,60,70,80,90,100]
result=pd.DataFrame()
for i in day:
    result=result.append( pd.DataFrame([getStratStats_single(back_test_buy_and_hold(stock_sub,'to_keep','ret',holding=i)['strat_ret'])],index=['Hold_'+str(i)]))
result=result.append( pd.DataFrame([getStratStats_single(back_test_buy_and_hold(stock_sub,'to_keep','ret',holding=i)['ret'])],index=['Buy&Hold']))
result

,tot_ret,annual_ret,annual_vol,sortino_ratio,sharpe_ratio,max_drawdown,max_drawdown_duration
Hold_10,-0.166289,-0.031223,0.030904,-0.343807,-1.657479,0.156187,2024
Hold_20,-0.157092,-0.029368,0.050099,-0.304322,-0.985406,0.184031,2024
Hold_30,-0.177850,-0.033580,0.061924,-0.348815,-0.865258,0.200634,2024
Hold_40,-0.236446,-0.045963,0.075571,-0.374501,-0.872869,0.249475,2024
Hold_50,-0.201631,-0.038515,0.079640,-0.351036,-0.734746,0.244810,2024
Hold_60,-0.311252,-0.062967,0.088876,-0.506140,-0.933518,0.339565,2024
Hold_70,-0.376619,-0.079124,0.099110,-0.570472,-1.000137,0.415136,2024
Hold_80,-0.467111,-0.103974,0.108713,-0.680338,-1.140385,0.482671,2024
Hold_90,-0.492510,-0.111574,0.127212,-0.648390,-1.034292,0.555396,2024
Hold_100,-0.471090,-0.105145,0.135336,-0.634835,-0.924701,0.551637,2024


In [180]:
stock_sub=stock.loc[df.index[2]:str(df.index[2].year+5)+"-"+"12-31"]
stock_sub=stock_sub.join(df)
stock_sub['to_keep']=stock_sub['to_keep'].fillna(False)#true=signal day

day=[10,20,30,40,50,60,70,80,90,100]
result=pd.DataFrame()
for i in day:
    result=result.append( pd.DataFrame([getStratStats_single(back_test_buy_and_hold(stock_sub,'to_keep','ret',holding=i)['strat_ret'])],index=['Hold_'+str(i)]))
result=result.append( pd.DataFrame([getStratStats_single(back_test_buy_and_hold(stock_sub,'to_keep','ret',holding=i)['ret'])],index=['Buy&Hold']))
result

,tot_ret,annual_ret,annual_vol,sortino_ratio,sharpe_ratio,max_drawdown,max_drawdown_duration
Hold_10,-0.127854,-0.023561,0.044743,-0.197181,-0.973573,0.191516,2026
Hold_20,-0.055870,-0.009970,0.054221,-0.147587,-0.552743,0.151576,2026
Hold_30,-0.025430,-0.004479,0.062355,-0.134679,-0.392583,0.162263,2026
Hold_40,-0.094496,-0.017152,0.073914,-0.188251,-0.502637,0.230339,2026
Hold_50,-0.030329,-0.005353,0.078336,-0.136690,-0.323651,0.200284,2026
Hold_60,-0.194164,-0.036926,0.085586,-0.307547,-0.665135,0.248078,2026
Hold_70,-0.193770,-0.036844,0.093163,-0.314863,-0.610159,0.235161,2026
Hold_80,-0.276912,-0.054943,0.096617,-0.433943,-0.775664,0.319593,2026
Hold_90,-0.261910,-0.051554,0.107600,-0.406732,-0.665002,0.307805,2026
Hold_100,-0.202597,-0.038690,0.111260,-0.340758,-0.527508,0.307805,2026


In [181]:
stock_sub=stock.loc[df.index[3]:str(df.index[3].year+5)+"-"+"12-31"]
stock_sub=stock_sub.join(df)
stock_sub['to_keep']=stock_sub['to_keep'].fillna(False)#true=signal day

day=[10,20,30,40,50,60,70,80,90,100]
result=pd.DataFrame()
for i in day:
    result=result.append( pd.DataFrame([getStratStats_single(back_test_buy_and_hold(stock_sub,'to_keep','ret',holding=i)['strat_ret'])],index=['Hold_'+str(i)]))
result=result.append( pd.DataFrame([getStratStats_single(back_test_buy_and_hold(stock_sub,'to_keep','ret',holding=i)['ret'])],index=['Buy&Hold']))
result

,tot_ret,annual_ret,annual_vol,sortino_ratio,sharpe_ratio,max_drawdown,max_drawdown_duration
Hold_10,-0.043512,-0.008385,0.041139,-0.115540,-0.689977,0.135716,1869
Hold_20,0.009395,0.001771,0.044622,-0.089091,-0.408507,0.115312,1869
Hold_30,0.074472,0.013688,0.047866,-0.034570,-0.131863,0.098773,1016
Hold_40,0.071569,0.013169,0.051153,-0.041891,-0.133535,0.127398,996
Hold_50,0.088176,0.016123,0.055058,-0.025186,-0.070418,0.140759,997
Hold_60,0.005130,0.000969,0.063143,-0.109499,-0.301397,0.132068,970
Hold_70,0.012459,0.002346,0.067301,-0.105661,-0.262309,0.174239,794
Hold_80,-0.011261,-0.002141,0.069941,-0.140212,-0.316572,0.176360,939
Hold_90,-0.080227,-0.015704,0.076698,-0.219255,-0.465514,0.226395,940
Hold_100,-0.094174,-0.018547,0.080966,-0.230576,-0.476082,0.227794,940


In [177]:
stock_sub=stock.loc[df.index[4]:str(df.index[4].year+5)+"-"+"12-31"]
stock_sub=stock_sub.join(df)
stock_sub['to_keep']=stock_sub['to_keep'].fillna(False)#true=signal day

day=[10,20,30,40,50,60,70,80,90,100]
result=pd.DataFrame()
for i in day:
    result=result.append( pd.DataFrame([getStratStats_single(back_test_buy_and_hold(stock_sub,'to_keep','ret',holding=i)['strat_ret'])],index=['Hold_'+str(i)]))
result=result.append( pd.DataFrame([getStratStats_single(back_test_buy_and_hold(stock_sub,'to_keep','ret',holding=i)['ret'])],index=['Buy&Hold']))
result

,tot_ret,annual_ret,annual_vol,sortino_ratio,sharpe_ratio,max_drawdown,max_drawdown_duration
Hold_10,-0.016788,-0.003041,0.039827,-0.086248,-0.578532,0.056419,1308
Hold_20,0.041192,0.007289,0.043893,-0.057755,-0.289598,0.056419,1293
Hold_30,0.121827,0.020898,0.048207,0.004709,0.018618,0.056419,1273
Hold_40,0.132568,0.022649,0.050446,0.015588,0.052518,0.058061,1271
Hold_50,0.135552,0.023133,0.054021,0.020068,0.058004,0.056419,611
Hold_60,0.009319,0.001670,0.061861,-0.105335,-0.296309,0.160426,1887
Hold_70,-0.041504,-0.007597,0.066669,-0.167169,-0.413944,0.175858,1887
Hold_80,-0.009127,-0.001648,0.069127,-0.135779,-0.313167,0.172104,1887
Hold_90,-0.060743,-0.011211,0.076442,-0.188781,-0.408296,0.224721,1887
Hold_100,-0.176340,-0.034300,0.085106,-0.299478,-0.638027,0.277214,1887


In [182]:
stock_sub=stock.loc[df.index[5]:str(df.index[5].year+5)+"-"+"12-31"]
stock_sub=stock_sub.join(df)
stock_sub['to_keep']=stock_sub['to_keep'].fillna(False)#true=signal day

day=[10,20,30,40,50,60,70,80,90,100]
result=pd.DataFrame()
for i in day:
    result=result.append( pd.DataFrame([getStratStats_single(back_test_buy_and_hold(stock_sub,'to_keep','ret',holding=i)['strat_ret'])],index=['Hold_'+str(i)]))
result=result.append( pd.DataFrame([getStratStats_single(back_test_buy_and_hold(stock_sub,'to_keep','ret',holding=i)['ret'])],index=['Buy&Hold']))
result

,tot_ret,annual_ret,annual_vol,sortino_ratio,sharpe_ratio,max_drawdown,max_drawdown_duration
Hold_10,0.031765,0.005281,0.017969,-0.069869,-0.819138,0.008170,1154
Hold_20,0.068796,0.011269,0.021451,-0.059431,-0.407033,0.019851,1142
Hold_30,0.115464,0.018574,0.025547,-0.012090,-0.055825,0.030888,14
Hold_40,0.142716,0.022723,0.026920,0.026849,0.101154,0.030888,1118
Hold_50,0.097065,0.015725,0.029330,-0.047216,-0.145768,0.037812,1135
Hold_60,0.061938,0.010173,0.033740,-0.104013,-0.291266,0.056476,1092
Hold_70,0.058483,0.009618,0.037565,-0.109742,-0.276365,0.063588,1126
Hold_80,0.088588,0.014399,0.039253,-0.061850,-0.142703,0.068485,1105
Hold_90,0.058871,0.009681,0.042583,-0.113886,-0.242333,0.068485,1105
Hold_100,-0.055427,-0.009558,0.051332,-0.245033,-0.575816,0.143250,2039
